# Mathematical Underpinnings - Lab 6

In [1]:
from sklearn.metrics import mutual_info_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tqdm import tqdm

## Useful functions

In [2]:
def discetize_2bins(X):
    X_discrete = 1*(X >= 0)
    return X_discrete

In [3]:
def conditional_permutation(X, Z):

    z_values = np.unique(Z)
    n_z_values = len(z_values)
    n = len(Z)

    X_b = np.zeros(n)

    for i in range(n_z_values):

        z_value_tmp = z_values[i]

        X_b[Z == z_value_tmp] = np.random.permutation(X[Z == z_value_tmp])

    return X_b

In [4]:
def conditional_mutual_information(X, Y, Z):

    z_values = np.unique(Z)
    n_z_values = len(z_values)
    n = len(Z)

    cmi = 0

    for i in range(n_z_values):

        z_value_tmp = z_values[i]
        z_condition = (Z == z_value_tmp)

        X_z = X[z_condition]
        Y_z = Y[z_condition]

        mi_XY_z = mutual_info_score(X_z, Y_z)
        p_z = np.sum(z_condition)/n

        cmi += p_z*mi_XY_z

    return cmi

In [5]:
# II(X;Y;Z)
def interaction_information(X, Y, Z):
    return conditional_mutual_information(X, Y, Z) - mutual_info_score(X, Y)

In [6]:
# II(X;Y;Z1;Z2)
def interaction_information2(X, Y, Z1, Z2):
    Z_1_and_2 = 2*Z2 + Z1
    return interaction_information(X, Y, Z_1_and_2) - interaction_information(X, Y, Z1) - interaction_information(X, Y, Z2)

## Task 1

In [7]:
def secmi2(X, Y, Z):    
    inter_info_sum = 0
    for i in range(Z.shape[1]):
        inter_info_sum += interaction_information(X, Y, Z[:, i])
    
    return mutual_info_score(X, Y) + inter_info_sum

In [8]:
def secmi3(X, Y, Z):
    inter2_info_sum = 0
    for i in range(Z.shape[1]):
        for j in range(i+1, Z.shape[1]):
            inter2_info_sum += interaction_information2(X, Y, Z[:, i], Z[:, j])
    return secmi2(X, Y, Z) + inter2_info_sum

### a)

In [9]:
def cond_indep_test_permutation(X, Y, Z, B, stat):

    n_col_Z = Z.shape[1]
    Z_1dim = np.dot(Z, 2**np.linspace(0, n_col_Z-1, n_col_Z))

    if stat == "cmi":
        stat_value = conditional_mutual_information(X, Y, Z_1dim)
    if stat == "secmi2":
        stat_value = secmi2(X, Y, Z)
    if stat == "secmi3":
        stat_value = secmi3(X, Y, Z)

    condition_p_value = 0
    for b in range(B):
        X_b = conditional_permutation(X, Z_1dim)

        if stat == "cmi":
            stat_value_b = conditional_mutual_information(X_b, Y, Z_1dim)
        if stat == "secmi2":
            stat_value_b = secmi2(X_b, Y, Z)
        if stat == "secmi3":
            stat_value_b = secmi3(X_b, Y, Z)

        if stat_value <= stat_value_b:
            condition_p_value += 1

    p_value = (1 + condition_p_value)/(1 + B)

    return 2*len(X)*stat_value, p_value

### b)

In [10]:
def generate_sample(num_samples):
    n = num_samples

    Y = discetize_2bins(np.random.standard_normal(n))

    Z1 = discetize_2bins(np.random.standard_normal(n) + Y)
    Z2 = discetize_2bins(np.random.standard_normal(n) + Y)
    Z3 = discetize_2bins(np.random.standard_normal(n) + Y)

    X = discetize_2bins(np.random.standard_normal(n) + Z1)

    return X, Y, Z1, Z2, Z3

#### Test 1

In [11]:
rejected_cmi = 0
rejected_secmi2 = 0
rejected_secmi3 = 0

for i in tqdm(range(100), total=100):
    X, Y, Z1, Z2, Z3 = generate_sample(1000)
    Z_1_2 = np.array([Z1, Z2]).T
    if cond_indep_test_permutation(X, Y, Z_1_2, B=50, stat="cmi")[1] < 0.05:
        rejected_cmi += 1
    if cond_indep_test_permutation(X, Y, Z_1_2, B=50, stat="secmi2")[1] < 0.05:
        rejected_secmi2 += 1
    if cond_indep_test_permutation(X, Y, Z_1_2, B=50, stat="secmi3")[1] < 0.05:
        rejected_secmi3 += 1

print(f"CMI rejection rate: {rejected_cmi/100}")
print(f"SECMI2 rejection rate: {rejected_secmi2/100}")
print(f"SECMI3 rejection rate: {rejected_secmi3/100}")

100%|██████████| 100/100 [01:42<00:00,  1.02s/it]

CMI rejection rate: 0.04
SECMI2 rejection rate: 0.04
SECMI3 rejection rate: 0.02


#### Test 2

In [12]:
rejected_cmi = 0
rejected_secmi2 = 0
rejected_secmi3 = 0

for i in tqdm(range(100), total=100):
    X, Y, Z1, Z2, Z3 = generate_sample(1000)
    Z_2_3 = np.array([Z2, Z3]).T
    if cond_indep_test_permutation(X, Y, Z_2_3, B=50, stat="cmi")[1] < 0.05:
        rejected_cmi += 1
    if cond_indep_test_permutation(X, Y, Z_2_3, B=50, stat="secmi2")[1] < 0.05:
        rejected_secmi2 += 1
    if cond_indep_test_permutation(X, Y, Z_2_3, B=50, stat="secmi3")[1] < 0.05:
        rejected_secmi3 += 1

print(f"CMI rejection rate: {rejected_cmi/100}")
print(f"SECMI2 rejection rate: {rejected_secmi2/100}")
print(f"SECMI3 rejection rate: {rejected_secmi3/100}")

100%|██████████| 100/100 [01:40<00:00,  1.00s/it]

CMI rejection rate: 0.8
SECMI2 rejection rate: 0.85
SECMI3 rejection rate: 0.82


### c)

In [13]:
def generate_sample_2(num_samples=1000):
    n = num_samples

    X = np.random.binomial(1, 0.5, n)
    Z1 = np.random.binomial(1, 0.5, n)
    Z2 = np.random.binomial(1, 0.5, n)
    Z3 = np.random.binomial(1, 0.5, n)

    M = (X + Z1 + Z2) % 2

    ind_0 = np.where(M == 0)[0]
    ind_1 = np.where(M == 1)[0]

    Y = np.zeros(n)
    Y[ind_0] = np.random.binomial(1, 0.2, len(ind_0))
    Y[ind_1] = np.random.binomial(1, 0.8, len(ind_1))

    return X, Y, Z1, Z2, Z3

#### Test 1

In [14]:
rejected_cmi = 0
rejected_secmi2 = 0
rejected_secmi3 = 0

for i in tqdm(range(100), total=100):
    X, Y, Z1, Z2, Z3 = generate_sample_2(1000)
    Z_1_2 = np.array([Z1, Z2]).T
    if cond_indep_test_permutation(X, Y, Z_1_2, B=50, stat="cmi")[1] < 0.05:
        rejected_cmi += 1
    if cond_indep_test_permutation(X, Y, Z_1_2, B=50, stat="secmi2")[1] < 0.05:
        rejected_secmi2 += 1
    if cond_indep_test_permutation(X, Y, Z_1_2, B=50, stat="secmi3")[1] < 0.05:
        rejected_secmi3 += 1

print(f"CMI rejection rate: {rejected_cmi/100}")
print(f"SECMI2 rejection rate: {rejected_secmi2/100}")
print(f"SECMI3 rejection rate: {rejected_secmi3/100}")

100%|██████████| 100/100 [01:44<00:00,  1.04s/it]

CMI rejection rate: 1.0
SECMI2 rejection rate: 0.0
SECMI3 rejection rate: 1.0


#### Test 2

In [15]:
rejected_cmi = 0
rejected_secmi2 = 0
rejected_secmi3 = 0

for i in tqdm(range(100), total=100):
    X, Y, Z1, Z2, Z3 = generate_sample_2(1000)
    Z_2_3 = np.array([Z2, Z3]).T
    if cond_indep_test_permutation(X, Y, Z_2_3, B=50, stat="cmi")[1] < 0.05:
        rejected_cmi += 1
    if cond_indep_test_permutation(X, Y, Z_2_3, B=50, stat="secmi2")[1] < 0.05:
        rejected_secmi2 += 1
    if cond_indep_test_permutation(X, Y, Z_2_3, B=50, stat="secmi3")[1] < 0.05:
        rejected_secmi3 += 1

print(f"CMI rejection rate: {rejected_cmi/100}")
print(f"SECMI2 rejection rate: {rejected_secmi2/100}")
print(f"SECMI3 rejection rate: {rejected_secmi3/100}")

100%|██████████| 100/100 [01:43<00:00,  1.03s/it]

CMI rejection rate: 0.02
SECMI2 rejection rate: 0.02
SECMI3 rejection rate: 0.03


## Task 2
 
in R